In [1]:
import pandas as pd
import spacy
nlp=spacy.load("en_core_web_sm")

In [2]:
ds=pd.read_csv('smsspamcollection.tsv',sep='\t')

In [3]:
ds['label']=ds['label'].replace('ham',1)
ds['label']=ds['label'].replace('spam',0)

In [6]:
y=ds['label']
X=ds['message']

In [7]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=0,test_size=0.2)

In [11]:
X_test.shape

(1115,)

In [12]:
def sent_tokenizer(sentence):
    doc_sent=nlp(sentence)
    doc_ds=[]
    for token in doc_sent:
        if token.pos_ not in {'PUNCT','SYM','NUM'} and nlp.vocab[token.text].is_stop == False:
            lem_k=token.lemma_
            doc_ds.append(lem_k.lower())
    return doc_ds


In [13]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier

In [14]:
classifier=RandomForestClassifier(n_estimators=100, random_state=0, criterion='gini')

In [15]:
tfidf=TfidfVectorizer(tokenizer=sent_tokenizer)

In [17]:
clf=Pipeline([('tft',tfidf),('classi',classifier)])

In [19]:
clf.fit(X_train,y_train)

Pipeline(memory=None,
     steps=[('tft', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
 ...mators=100, n_jobs=None,
            oob_score=False, random_state=0, verbose=0, warm_start=False))])

In [20]:
y_pred=clf.predict(X_test)

In [27]:
from sklearn.metrics import confusion_matrix, consensus_score, accuracy_score,completeness_score,classification_report

In [56]:
cm=confusion_matrix(y_test,y_pred,labels=[0,1])

In [57]:
cm

array([[143,  17],
       [  0, 955]])

In [28]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       1.00      0.89      0.94       160
           1       0.98      1.00      0.99       955

   micro avg       0.98      0.98      0.98      1115
   macro avg       0.99      0.95      0.97      1115
weighted avg       0.99      0.98      0.98      1115



In [29]:
print(accuracy_score(y_test,y_pred))

0.9847533632286996


In [65]:
clf.predict(['claim'])

array([0])